In [1]:
import os
import pathlib
import pprint
import json
import itertools
import shutil

import numpy as np
import pandas as pd

In [2]:
def save_json_dict(
        json_filename,
        json_dict
        ):

    with open(json_filename, 'w') as json_file:
        json.dump(json_dict, json_file, indent=4)


def load_json_dict(
        json_filename
        ):

    with open(json_filename, 'r') as json_file:
        json_dict = json.load(json_file)

    return json_dict

In [3]:
params_df = pd.DataFrame({
    "idx": pd.Series(dtype="int"),
    "cutdown_ratio": pd.Series(dtype="float"),
    "backbone_class": pd.Series(dtype="str"),
    "num_gpus": pd.Series(dtype="int"),
    "img_size": pd.Series(dtype="int"),
    "batch_size": pd.Series(dtype="int"),
    "autocast": pd.Series(dtype="bool"),
    "num_workers": pd.Series(dtype="int"),
    "exists": pd.Series(dtype="bool")
})

#

directory_list = [
    "efficientnet_v2_s",
    "swint_v2_t",
    "cvt_21-384-22k"
]

for directory in directory_list:    
    params_subdir = os.path.join(pathlib.Path.home(), "fashion_retrieval", "params", "ret_test_DDP_2", directory)
    for filename in os.listdir(params_subdir):
        full_filename = os.path.join(params_subdir, filename)
        params = load_json_dict(full_filename)

        exp_idx = int(filename[-31:-28])

        exp_path = os.path.join(
            pathlib.Path.home(),
            "data",
            "fashion_retrieval",
            "ret_test_DDP_2",
            "cvt_21-384-22k",
            directory,
            "test_{:03d}".format(exp_idx)
            )
        
        exists_b = os.path.exists(exp_path)

        new_row_dict = {
            "idx": exp_idx,
            "cutdown_ratio": params["settings"]["data_loading"]["cutdown_ratio"],
            "backbone_class": params["settings"]["backbone"]["class"],
            "num_gpus": len(params["settings"]["device_idxs"]),
            "img_size": params["settings"]["backbone"]["img_size"],
            "batch_size": params["settings"]["stage_1"]["data_loading"]["batch_size"],
            "autocast": params["settings"]["stage_1"]["autocast"]["enabled"],
            "num_workers": params["settings"]["stage_1"]["data_loading"]["num_workers"],
            "exists": exists_b
        }

        params_df.loc[len(params_df)] = new_row_dict

In [4]:
print(params_df["idx"].value_counts())
print(params_df["backbone_class"].value_counts())
print(params_df["num_gpus"].value_counts())
print(params_df["img_size"].value_counts())
print(params_df["batch_size"].value_counts())
print(params_df["autocast"].value_counts())
print(params_df["num_workers"].value_counts())
print(params_df["exists"].value_counts())

idx
145    3
31     3
146    3
91     3
54     3
      ..
151    3
42     3
179    3
118    3
134    3
Name: count, Length: 240, dtype: int64
backbone_class
EfficientNetV2SmallBackbone         240
SwinTransformerV2TinyBackbone       240
CvTransformerB21I384D22kBackbone    240
Name: count, dtype: int64
num_gpus
1    180
2    180
3    180
4    180
Name: count, dtype: int64
img_size
384    360
256    360
Name: count, dtype: int64
batch_size
8     144
32    144
4     144
16    144
64    144
Name: count, dtype: int64
autocast
True     360
False    360
Name: count, dtype: int64
num_workers
4    240
8    240
2    240
Name: count, dtype: int64
exists
False    720
Name: count, dtype: int64


In [6]:
params_df[
    (params_df["backbone_class"] == "CvTransformerB21I384D22kBackbone") &\
    (params_df["idx"].isin([216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239]))
]

,idx,cutdown_ratio,backbone_class,num_gpus,img_size,batch_size,autocast,num_workers,exists
492,230,0.02,CvTransformerB21I384D22kBackbone,3,384,64,True,8,False
509,220,0.02,CvTransformerB21I384D22kBackbone,1,384,64,False,4,False
522,236,0.02,CvTransformerB21I384D22kBackbone,4,384,64,True,8,False
525,228,0.02,CvTransformerB21I384D22kBackbone,3,384,64,True,2,False
528,227,0.02,CvTransformerB21I384D22kBackbone,2,384,64,False,8,False
535,235,0.02,CvTransformerB21I384D22kBackbone,4,384,64,True,4,False
540,238,0.02,CvTransformerB21I384D22kBackbone,4,384,64,False,4,False
548,229,0.02,CvTransformerB21I384D22kBackbone,3,384,64,True,4,False
574,218,0.02,CvTransformerB21I384D22kBackbone,1,384,64,True,8,False
596,239,0.02,CvTransformerB21I384D22kBackbone,4,384,64,False,8,False


---

In [46]:
backbone_to_directory_dict = {
    "EfficientNetV2SmallBackbone": "efficientnet_v2_s",
    "SwinTransformerV2TinyBackbone": "swint_v2_t",
    "CvTransformerB21I384D22kBackbone": "cvt_21-384-22k"
}

---

In [47]:
ord_params_df = params_df[
    (params_df["num_gpus"].isin([3, 4]))
]

print(len(ord_params_df))

360


In [48]:
ord_params_df.head(5)

,idx,cutdown_ratio,backbone_class,num_gpus,img_size,batch_size,autocast,num_workers,exists
4,37,0.02,EfficientNetV2SmallBackbone,3,256,8,True,4,False
6,213,0.02,EfficientNetV2SmallBackbone,4,384,32,False,2,False
7,181,0.02,EfficientNetV2SmallBackbone,3,384,16,True,4,False
9,117,0.02,EfficientNetV2SmallBackbone,4,256,64,False,2,False
10,214,0.02,EfficientNetV2SmallBackbone,4,384,32,False,4,False


In [50]:
run_filename = "run_tests__all__3-4__0123.sh"
run_file = open(run_filename, "w")

for zidx, (index, row) in enumerate(ord_params_df.iterrows()):
    idx = row["idx"]
    directory = backbone_to_directory_dict[row["backbone_class"]]
    
    print("echo \"Running experiment {:s} {:03d} ({:d}/{:d})\"".format(directory, idx, zidx + 1, len(ord_params_df)), file=run_file)

    print("python train_ret_DDP_preview.py {:s} --master_port 12356 --reset_experiment --terminal_silent".format(
        os.path.join("params", "ret_test_DDP_2", directory, "test_{:03d}__train_ret_DDP_preview.json".format(idx))
    ), file=run_file)

    print("echo \"Finished experiment {:s} {:03d} ({:d}/{:d})\"".format(directory, idx, zidx + 1, len(ord_params_df)), file=run_file)

run_file.close()

---

In [51]:
ord_params_df = params_df[
    (params_df["num_gpus"].isin([2]))
]

print(len(ord_params_df))

180


In [52]:
ord_params_df.head(5)

,idx,cutdown_ratio,backbone_class,num_gpus,img_size,batch_size,autocast,num_workers,exists
1,31,0.02,EfficientNetV2SmallBackbone,2,256,8,True,4,False
8,7,0.02,EfficientNetV2SmallBackbone,2,256,4,True,4,False
13,128,0.02,EfficientNetV2SmallBackbone,2,384,4,True,8,False
17,129,0.02,EfficientNetV2SmallBackbone,2,384,4,False,2,False
20,177,0.02,EfficientNetV2SmallBackbone,2,384,16,False,2,False


In [53]:
run_filename = "run_tests__all__2__12.sh"
run_file = open(run_filename, "w")

for zidx, (index, row) in enumerate(ord_params_df.iterrows()):
    idx = row["idx"]
    directory = backbone_to_directory_dict[row["backbone_class"]]
    
    print("echo \"Running experiment {:s} {:03d} ({:d}/{:d})\"".format(directory, idx, zidx + 1, len(ord_params_df)), file=run_file)

    print("python train_ret_DDP_preview.py {:s} --master_port 12356 --reset_experiment --terminal_silent".format(
        os.path.join("params", "ret_test_DDP_2", directory, "test_{:03d}__train_ret_DDP_preview.json".format(idx))
    ), file=run_file)

    print("echo \"Finished experiment {:s} {:03d} ({:d}/{:d})\"".format(directory, idx, zidx + 1, len(ord_params_df)), file=run_file)

run_file.close()

---

In [54]:
ord_params_df = params_df[
    (params_df["num_gpus"].isin([1]))
]

print(len(ord_params_df))

180


In [55]:
ord_params_df.head(5)

,idx,cutdown_ratio,backbone_class,num_gpus,img_size,batch_size,autocast,num_workers,exists
0,145,0.02,EfficientNetV2SmallBackbone,1,384,8,True,4,False
2,77,0.02,EfficientNetV2SmallBackbone,1,256,32,False,8,False
3,121,0.02,EfficientNetV2SmallBackbone,1,384,4,True,4,False
5,147,0.02,EfficientNetV2SmallBackbone,1,384,8,False,2,False
16,27,0.02,EfficientNetV2SmallBackbone,1,256,8,False,2,False


In [56]:
run_filename = "run_tests__all__1__0.sh"
run_file = open(run_filename, "w")

for zidx, (index, row) in enumerate(ord_params_df.iterrows()):
    idx = row["idx"]
    directory = backbone_to_directory_dict[row["backbone_class"]]
    
    print("echo \"Running experiment {:s} {:03d} ({:d}/{:d})\"".format(directory, idx, zidx + 1, len(ord_params_df)), file=run_file)

    print("python train_ret_DDP_preview.py {:s} --master_port 12356 --reset_experiment --terminal_silent".format(
        os.path.join("params", "ret_test_DDP_2", directory, "test_{:03d}__train_ret_DDP_preview.json".format(idx))
    ), file=run_file)

    print("echo \"Finished experiment {:s} {:03d} ({:d}/{:d})\"".format(directory, idx, zidx + 1, len(ord_params_df)), file=run_file)

run_file.close()

---

In [10]:
ord_params_df = params_df[
    (params_df["backbone_class"] == "CvTransformerB21I384D22kBackbone") &\
    (params_df["idx"].isin([216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239]))
]

In [11]:
ord_params_df

,idx,cutdown_ratio,backbone_class,num_gpus,img_size,batch_size,autocast,num_workers,exists
492,230,0.02,CvTransformerB21I384D22kBackbone,3,384,64,True,8,False
509,220,0.02,CvTransformerB21I384D22kBackbone,1,384,64,False,4,False
522,236,0.02,CvTransformerB21I384D22kBackbone,4,384,64,True,8,False
525,228,0.02,CvTransformerB21I384D22kBackbone,3,384,64,True,2,False
528,227,0.02,CvTransformerB21I384D22kBackbone,2,384,64,False,8,False
535,235,0.02,CvTransformerB21I384D22kBackbone,4,384,64,True,4,False
540,238,0.02,CvTransformerB21I384D22kBackbone,4,384,64,False,4,False
548,229,0.02,CvTransformerB21I384D22kBackbone,3,384,64,True,4,False
574,218,0.02,CvTransformerB21I384D22kBackbone,1,384,64,True,8,False
596,239,0.02,CvTransformerB21I384D22kBackbone,4,384,64,False,8,False


In [137]:
run_filename = "run_tests__cvt_21-384-22k__BIG.sh"
run_file = open(run_filename, "w")

for zidx, (index, row) in enumerate(ord_params_df.iterrows()):
    idx = row["idx"]
    
    print("echo \"Running cvt_21-384-22k experiment  {:03d} ({:d}/{:d})\"".format(idx, zidx, len(ord_params_df)), file=run_file)

    print("python train_ret_DDP_preview.py {:s} --master_port 12355 --reset_experiment --terminal_silent".format(
        os.path.join("params", "ret_test_DDP_2", "cvt_21-384-22k", "test_{:03d}__train_ret_DDP_preview.json".format(idx))
    ), file=run_file)

    print("echo \"Finished cvt_21-384-22k experiment {:03d} ({:d}/{:d})\"".format(idx, zidx + 1, len(ord_params_df)), file=run_file)

run_file.close()

---